In [ ]:
#Firstly select the correct graphs (100e or above) and get the data for it (in the correct form for the next function)
#Must enter in the correct order- smallest to largest

from ROOT import TCanvas, TGraph, TFile, TH1F, TH2D
import numpy as np
import re
import scipy as sc
import matplotlib.pyplot as plt


def collect_pulse_data(file):
    myFile = TFile.Open(file, "read")
    data = myFile.Get("PulseTransfer")
    detector = data.Get("detector1")
    pulsed_array = []
    data_x = []
    data_y = []
    for key in detector.GetListOfKeys():
        name = key.GetName()
        graph = detector.Get(name)
        if type(graph) != TGraph:
            continue
        else:
            searching = re.search("current", graph.GetTitle())
            searching1 = re.search(" = ", graph.GetTitle())
            if searching != None and name[:10] == "current_ev":
                a = graph.GetTitle()[searching1.end():searching1.end()+4]
                pulsed_array.append(a)
                event = detector.Get(name)
                x = event.GetX()
                y = event.GetY()
                data_x.append(x)
                data_y.append(y)
    
    return pulsed_array, data_x, data_y
    

#Code to firstly find the peaks of the fits for the pulses

def fit_convolve_data(x_data, y_data):
    error = []
    a=1000000000000000
    for x, y in zip(x_data, y_data): 
        print(len(x))
        for i in x:
            #print(len(np.array(i)))
            if a > len(np.array(i)):
                a = len(np.array(i))-1
                print(a)
        
        overall_std = [] 
        for p in range(a-1):
            std = []
            mean = (np.array(y[0])[:a][p] + np.array(y[1])[:a][p] + np.array(y[2])[:a][p] + np.array(y[3])[:a][p] + np.array(y[4])[:a][p] + np.array(y[5])[:a][p] + np.array(y[6])[:a][p] + np.array(y[7])[:a][p] + np.array(y[8])[:a][p])/10 #+ np.array(y[9])[:a][p])/10
            for i in y:
                diff = (mean - np.array(i)[p])**2
                std.append(diff)
            
            sigma = np.sqrt(sum(std)/(len(std)-1))
            if sigma < 1e-4:
                sigma = 1e-4
            overall_std.append(sigma)
    
    error.append(overall_std)
               
    return error

files = ['root_files/1_test_-5.root', 'root_files/1_test_-10.root', 'root_files/1_test_-15.root', 'root_files/1_test_-20.root', 'root_files/1_test_-25.root', 'root_files/1_test_-30.root', 'root_files/1_test_-35.root', 'root_files/1_test_-40.root']
x_values, y_values, pulse_names, hole_dat = [], [], [], []
for dat in files:
    pulse, x_dat, y_dat = collect_pulse_data(dat)
    pulse_names.append(pulse)
    x_values.append(x_dat)
    y_values.append(y_dat)


print(y_values)
overall_std = fit_convolve_data(x_values, y_values) 
print(overall_std)
print(len(overall_std))